In [ ]:
!launch_scripts/baseline.sh "photo,cartoon,sketch"

In [2]:
import os

def run_experiment(category, configurations):
  for configuration in configurations:
    alpha = configuration["a"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    !launch_scripts/activation_shaping_experiments.sh {category} "{extension}" {alpha} {layers}
          

c = [
  {"a": 0.9, "LAYERS": "3"}
]
run_experiment("photo,cartoon,sketch", c)

Applied to 1 layers
100%|███████████████████████████████████████████| 31/31 [00:02<00:00, 12.91it/s]
corrupted size vs. prev_size
launch_scripts/activation_shaping_experiments.sh: line 12: 79870 Aborted                 python3 main.py --experiment=activation_shaping_experiments --experiment_name=activation_shaping_experiments/${target_domain}/ --dataset_args="{'root': 'data/PACS', 'source_domain': 'art_painting', 'target_domain': '${target_domain}','ALPHA': ${ALPHA}, 'LAYERS_LIST': '${LAYERS_LIST}'}" --batch_size=128 --num_workers=5 --grad_accum_steps=1 ${test_only}


In [ ]:
#import plotly.graph_objects as go
import numpy as np
import re

#Parse outputs from previous step and produce some statistics
def get_accuracy(EXP_TARGET, ALPHA, APPLY_EVERY_N, SKIP_FIRST_N):
  def get_max_accuracy_from_file(file_path):
    max_a_binarization = {}
    max_a_no_binarization = {}
    with open(file_path) as file:
      lines = file.readlines()
      for line in lines:
        pattern = r'TEST WITH BINARIZATION Evaluate: (\w+): Accuracy: (\d+\.\d+) - Loss: (\d+\.\d+)'
        match = re.match(pattern, line)
        if match:
            category = match.group(1)
            accuracy = float(match.group(2))
            if (category in max_a_binarization):
              max_a_binarization[category] = max(accuracy, max_a_binarization[category])
            else:
              max_a_binarization[category] = accuracy
        else:
            pattern = r'TEST WITHOUT BINARIZATION Evaluate: (\w+): Accuracy: (\d+\.\d+) - Loss: (\d+\.\d+)'
            match = re.match(pattern, line)
            if match:
                category = match.group(1)
                accuracy = float(match.group(2))
                if (category in max_a_no_binarization):
                    max_a_no_binarization[category] = max(accuracy, max_a_no_binarization[category])
                else:
                    max_a_no_binarization[category] = accuracy
    return max_a_binarization, max_a_no_binarization
  LOG_FILENAME = f"SKIP_{SKIP_FIRST_N}_ALPHA_{ALPHA}-APPLY_EVERY_N_{APPLY_EVERY_N}-log.txt"
  LOG_PATH = os.path.join(f"record/activation_shaping_experiments/{EXP_TARGET}", LOG_FILENAME)
  return get_max_accuracy_from_file(LOG_PATH)


class Results:
  def __init__(self, target):
    self.results_bin = {}
    self.results_no_bin = {}
    self.target = target

  def add_result(self, alpha, apply_every, skip_first):
    accuracy_bin, accuracy_no_bin = get_accuracy(self.target, alpha, apply_every, skip_first)
    
    for category in accuracy_bin.keys():
      if not category in self.results_bin:
        self.results_bin[category] = []
      self.results_bin[category].append({"accuracy": accuracy_bin[category], "alpha": alpha, "apply_every":apply_every, "skip_first": skip_first})
    for category in accuracy_no_bin.keys():
      if not category in self.results_no_bin:
        self.results_no_bin[category] = []
      self.results_no_bin[category].append({"accuracy": accuracy_no_bin[category], "alpha": alpha, "apply_every":apply_every, "skip_first": skip_first})

  def get_results_bin(self, category, alpha=None, apply_every=None, skip_first=None):
    to_return = []
    for result in self.results_bin[category]:
      if ((alpha is None or result["alpha"] == alpha)
      and (apply_every is None or result["apply_every"] == apply_every)
      and (skip_first is None or result["skip_first"] == skip_first)):
        to_return.append(result)
    return to_return
  def get_results_no_bin(self, category, alpha=None, apply_every=None, skip_first=None):
    to_return = []
    for result in self.results_no_bin[category]:
      if ((alpha is None or result["alpha"] == alpha)
      and (apply_every is None or result["apply_every"] == apply_every)
      and (skip_first is None or result["skip_first"] == skip_first)):
        to_return.append(result)
    return to_return


alphas = [0.9, 0.7, 0.5]
apply_every_n_list = [0, 2]
skip_first_n_list = [15, 14, 13, 10, 6 ]
categories = ["photo","cartoon","sketch"]
results = Results("photo,cartoon,sketch")
for alpha in alphas:
  for apply_every in apply_every_n_list:
    for skip_first in skip_first_n_list:
      results.add_result(alpha, apply_every, skip_first)

"""def make_plot(plot_title, raw_results, axis_values):
  y_max =[max(result, key=lambda r: r["accuracy"]) for result in raw_results]
  y_mean = [sum(result, key=lambda r: r["accuracy"]) / len(result) for result in raw_results]
  # Create traces
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=axis_values, y=y_max,
                      mode='lines',
                      name=f"{plot_title} - mean"))
  fig.add_trace(go.Scatter(x=axis_values, y=y_mean,
                      mode='lines',
                      name=f"{plot_title} - avg"))
  fig.show()"""

#make_plot("By alpha", [results.get_results("photos", alpha=a) for a in alphas])

for category in categories:
    print(f"\nResults for {category} - with binarization in the test mode")
    #Get max accuracy
    max_accuracy = max(results.get_results_bin(category), key=lambda r: r["accuracy"])
    print(f"Max accuracy: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by alpha
    for a in alphas:
        max_accuracy = max(results.get_results_bin(category, alpha=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for a = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by apply_every
    for a in apply_every_n_list:
        max_accuracy = max(results.get_results_bin(category, apply_every=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for apply_every = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by skip_first_n_list
    for a in skip_first_n_list:
        max_accuracy = max(results.get_results_bin(category, skip_first=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for skip_first_n_list = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")

    print(f"\nResults for {category} - without binarization in the test mode")
    #Get max accuracy
    max_accuracy = max(results.get_results_no_bin(category), key=lambda r: r["accuracy"])
    print(f"Max accuracy: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by alpha
    for a in alphas:
        max_accuracy = max(results.get_results_no_bin(category, alpha=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for a = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by apply_every
    for a in apply_every_n_list:
        max_accuracy = max(results.get_results_no_bin(category, apply_every=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for apply_every = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by skip_first_n_list
    for a in skip_first_n_list:
        max_accuracy = max(results.get_results_no_bin(category, skip_first=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for skip_first_n_list = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")


In [ ]:
#DOMAIN ADAPTATION STANDARD

import os
def check_if_experiment_already_run(K, LAYERS, EXP_TARGET, RECORD_MODE):
  LOG_FILENAME = f"L_{LAYERS}__K__{K}__RECORD_MODE__{RECORD_MODE}-log.txt"
  LOG_PATH = os.path.join(f"record/domain_adaptation/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    if (not check_if_experiment_already_run(K, layers, category, record_mode)):
        !launch_scripts/domain_adaptation.sh {category} "{extension}" {K} {record_mode} {layers}
    else:
        print("Already existing")
   

"""configurations = [
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "1"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "2"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "3"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "4"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "5"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "6"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "8"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "11"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,3"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "3,11"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "3,4"}
]
run_experiment("cartoon", configurations)"""
configurations = [
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "3"},
   {"K": -0.3, "RECORD_MODE": "threshold", "LAYERS": "3"},
]
for d in ["sketch"]:
  run_experiment(d, configurations)
"""BEST:
Cartoon: 70
Photo: 95
Sketch: 48 - 51 con K=-0,3
"""

In [17]:

import os
def check_if_experiment_already_run(K, APPLY_EVERY_N, SKIP_FIRST_N, EXP_TARGET, RECORD_MODE):
  LOG_FILENAME = f"SKIP_{SKIP_FIRST_N}_K_{K}-APPLY_EVERY_N_{APPLY_EVERY_N}-RECORD_MODE{RECORD_MODE}-log.txt"
  LOG_PATH = os.path.join(f"record/domain_adaptation_alt/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, configurations):
  for configuration in configurations:
    apply_every_n = configuration["apply_every_n"]
    skip_first_n = configuration["skip_first_n"]
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    if (not check_if_experiment_already_run(K, apply_every_n, skip_first_n, category, record_mode)):
        !launch_scripts/domain_adaptation_alt.sh {category} "" {K} {apply_every_n} {skip_first_n} {record_mode}
    else:
        print("Already existing")

          

#threshold
configurations = [
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "1"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "2"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "3"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "4"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "5"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "6"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "8"},
   {"K": 0, "RECORD_MODE": "threshold", "LAYERS": "11"},
]
for k in [0.1, 0.2, 0.5, 0.8]:
    configurations.append({ "skip_first_n": 14, "K": k, "RECORD_MODE": "topk"})

run_experiment("cartoon", configurations)

  #TODO: applicando a troppi layer non migliora, forse migliora rendendo k più selettivo
  #TODO se questo approccio funziona, prova anche con topK
  #Si può anche pensare di applicarlo a + layers

KeyError: 'apply_every_n'

In [4]:
import os
def check_if_experiment_already_run(K, LAYERS, EXP_TARGET, RECORD_MODE):
  LOG_FILENAME = f"L_{LAYERS}__K__{K}__RECORD_MODE{RECORD_MODE}-log.txt"
  LOG_PATH = os.path.join(f"record/domain_generalization/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    if (not check_if_experiment_already_run(K, layers, category, record_mode)):
        !launch_scripts/domain_generalization.sh {category} "{extension}" {K} {record_mode} {layers}
    else:
        print("Already existing")


"""c = [
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "1"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "0"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,3"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,3,4"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,5"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,11"}
]"""
#BEST lr = 0.003; Best wd = 0.0003
c = [
   #{"EXTENSION": "--extra_str='STATS' --print_stats=1", "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,3,15"},
  #{"EXTENSION": "--extra_str='TEST_MACRO_LAYER' ", "K": 0, "RECORD_MODE": "threshold", "LAYERS": "0"},
  #{"EXTENSION": "--extra_str='TEST_MACRO_LAYER' ", "K": 0, "RECORD_MODE": "threshold", "LAYERS": "1"},
  {"EXTENSION": "--extra_str='TEST_MACRO_LAYER_ZEROED_ACT' ", "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2"},
  #{"EXTENSION": "--extra_str='TEST_MACRO_LAYER' ", "K": 0, "RECORD_MODE": "threshold", "LAYERS": "3"},
]
for domain in ["cartoon"]:
  run_experiment(domain, c)

"""Idee per altre estensioni:
>Test: usare 0_like invece di M

>Randomizzare le coppie!
>Applicare nei primi layer per tot epoche, poi togliere e poi applicare ad ultimi!
>Provare extension 2
"""
  
"""Comment about statistics:
    >Interesting to see that, in each layer, the average M is very similar, but the product of all Ms is different,
     and it gets smaller and smaller as we move to deeper layers.
     This is counterintuitive, we would expect first layers to be different (focus on low level stuff) and next layers
     more similar, as they focus on high level stuff.
     But this is probably naive, this is what we would like to see if the network generalized well, but of course it doesn't.
    
    >Now it is interesting to see if, after we apply out activation shaping to the first layers, the last layers
     actually get more similar. => IT IS ACTUALLY HAPPENING!!
     
    >Another interesting observation is that on layer 14 the various M are very very different, and small.
     The article linked by professor states that the penultimate layer is the one containing more
     domain-specific informations!"""

 85%|███████████████████████████████████▋      | 90/106 [00:11<00:02,  7.88it/s]
Traceback (most recent call last):
  File "/home/marco/Activation-Shaping-AML/main.py", line 255, in <module>
    main()
  File "/home/marco/Activation-Shaping-AML/main.py", line 197, in main
    train(model, data)
  File "/home/marco/Activation-Shaping-AML/main.py", line 105, in train
    total_loss[0] += loss.item()
KeyboardInterrupt


"Comment about statistics:\n    >Interesting to see that, in each layer, the average M is very similar, but the product of all Ms is different,\n     and it gets smaller and smaller as we move to deeper layers.\n     This is counterintuitive, we would expect first layers to be different (focus on low level stuff) and next layers\n     more similar, as they focus on high level stuff.\n     But this is probably naive, this is what we would like to see if the network generalized well, but of course it doesn't.\n    \n    >Now it is interesting to see if, after we apply out activation shaping to the first layers, the last layers\n     actually get more similar. => IT IS ACTUALLY HAPPENING!!\n     \n    >Another interesting observation is that on layer 14 the various M are very very different, and small.\n     The article linked by professor states that the penultimate layer is the one containing more\n     domain-specific informations!"

In [4]:
import os
def check_if_experiment_already_run(K, LAYERS, EXP_TARGET, RECORD_MODE):
  LOG_FILENAME = f"L_{LAYERS}__K__{K}__RECORD_MODE{RECORD_MODE}-log.txt"
  LOG_PATH = os.path.join(f"record/domain_generalization_ext/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if ("EXTENSION" in configuration):
      extension = configuration["EXTENSION"]
    else:
      extension = ""
    if (not check_if_experiment_already_run(K, layers, category, record_mode)):
        !launch_scripts/domain_generalization_ext.sh {category} "{extension}" {K} {record_mode} {layers}
    else:
        print("Already existing")


c = [
   #{"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2"},
   #{"apply_every_n": 2, "K": 0.3, "RECORD_MODE": "topk", "LAYERS": "2"},
   #{"apply_every_n": 2, "K": 0.5, "RECORD_MODE": "topk", "LAYERS": "2"},
   #{"apply_every_n": 2, "K": 0.2, "RECORD_MODE": "topk", "LAYERS": "2"},
   #{"apply_every_n": 2, "K": 0.4, "RECORD_MODE": "topk", "LAYERS": "2,3,4,5"},
   #{"apply_every_n": 2, "K": 0.6, "RECORD_MODE": "topk", "LAYERS": "2,3,4,5"},
   #{"apply_every_n": 2, "K": 0.5, "RECORD_MODE": "topk", "LAYERS": "2,15"},
   {"apply_every_n": 2, "K": 0.5, "RECORD_MODE": "topk", "LAYERS": "15"},
   {"apply_every_n": 2, "K": 0.5, "RECORD_MODE": "topk", "LAYERS": "11"},
]
for domain in ["cartoon"]:
  run_experiment(domain, c)


 17%|███████▏                                  | 18/106 [00:04<00:21,  4.07it/s]
Traceback (most recent call last):
  File "/home/marco/Activation-Shaping-AML/main.py", line 255, in <module>
    main()
  File "/home/marco/Activation-Shaping-AML/main.py", line 197, in main
    train(model, data)
  File "/home/marco/Activation-Shaping-AML/main.py", line 115, in train
    scaler.step(optimizer)
  File "/home/marco/.local/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py", line 416, in step
    retval = self._maybe_opt_step(optimizer, optimizer_state, *args, **kwargs)
  File "/home/marco/.local/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py", line 314, in _maybe_opt_step
    if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values()):
  File "/home/marco/.local/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py", line 314, in <genexpr>
    if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values()):
KeyboardInterrupt
